# Preparing data

To run the models, we need the following input: 
- X_train_res and y_train_res: resampled train data using SMOTE
- X_test and y_test 
- X_train_12: with only the measurements for stage 1 and 2 
- X_test_12: with only the measurements for stage 1 and 2 

Import libraries. 

In [9]:
# Import necessary libraries
import json
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, auc, precision_recall_curve, confusion_matrix
import pandas as pd
import logging
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import precision_recall_curve, auc
from sklearn.metrics import confusion_matrix 
from collections import Counter

Standardized_complete_dataset is the dataframe I created with all the extracted features from TS fresh. 

In [10]:
complete_dataset = pd.read_csv('/Users/dionnespaltman/Desktop/V4/standardized_complete_dataset.csv', sep=',')
complete_dataset.drop('Unnamed: 0', axis=1, inplace=True)
display(complete_dataset)

,ID,Sum_12,Sum_4567,VVR_1,VVR_2,Sum_456,VVR_group,Condition,Date,Gender,...,AU26_r__standard_deviation,AU26_r__maximum,AU26_r__mean,AU26_r__root_mean_square,AU45_r__sum_values,AU45_r__variance,AU45_r__standard_deviation,AU45_r__maximum,AU45_r__mean,AU45_r__root_mean_square
0,23,24.0,37.0,13.0,11.0,27.0,0,2,2020-08-01,2,...,-0.289518,0.458331,-0.574983,-0.554028,-0.333560,0.221977,0.345618,0.548923,0.603537,0.429001
1,24,23.0,37.0,12.0,11.0,28.0,0,2,2020-01-22,2,...,1.793425,0.458331,2.294469,2.317495,0.203487,-0.260024,-0.030426,0.619766,-0.659902,-0.281599
2,25,28.0,44.0,16.0,12.0,33.0,1,2,2020-05-02,2,...,0.557817,0.458331,0.196854,0.400873,-0.376228,0.033873,0.204639,-0.135892,0.093018,0.147314
3,26,30.0,37.0,15.0,15.0,29.0,0,1,2020-06-02,1,...,-0.394321,0.458331,-0.847716,-0.761866,-0.868821,-0.323805,-0.084215,0.541052,-1.092069,-0.441394
4,27,22.0,39.0,11.0,11.0,31.0,0,2,2020-06-02,1,...,1.682971,1.863336,-2.230621,0.556978,2.456784,1.071168,0.914462,0.651252,3.692845,1.996765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,140,16.0,32.0,8.0,8.0,24.0,0,3,2021-05-26,2,...,-0.033310,0.458331,-0.446575,-0.318795,0.861706,0.199480,0.329100,0.210451,0.318307,0.327148
100,142,20.0,34.0,11.0,9.0,26.0,0,3,2021-05-31,1,...,-0.853016,0.458331,-1.199405,-1.246095,-0.067533,-0.585183,-0.317237,-0.537335,-0.522513,-0.509273
101,144,24.0,35.0,12.0,12.0,27.0,0,3,2021-01-06,1,...,-0.565965,0.458331,-0.851589,-0.878346,-0.720364,-0.806836,-0.534298,-0.340549,-0.961821,-0.835709
102,145,20.0,37.0,11.0,9.0,28.0,0,1,2021-02-06,2,...,-0.485857,0.366881,-0.548002,-0.665071,1.190821,0.027513,0.199753,0.076637,-0.006872,0.113717


In [11]:
columns_to_drop = ['ID', 'Sum_12', 'Sum_4567', 'Sum_456', 'VVR_group', 'Condition', 'Gender', 'Date'] 

These are the columns I use to predict, so all my features. I need these as a list to establish my featurizer. 
I have 102 features from TS fresh and then I added the two VVR measurements from stage 1 and 2. 

In [12]:
columns_ts_fresh = ['AU01_r__sum_values', 'AU01_r__variance', 'AU01_r__standard_deviation', 'AU01_r__maximum', 
                    'AU01_r__mean', 'AU01_r__root_mean_square', 'AU02_r__sum_values', 'AU02_r__variance', 
                    'AU02_r__standard_deviation', 'AU02_r__maximum', 'AU02_r__mean', 'AU02_r__root_mean_square', 
                    'AU04_r__sum_values', 'AU04_r__variance', 'AU04_r__standard_deviation', 'AU04_r__maximum', 
                    'AU04_r__mean', 'AU04_r__root_mean_square', 'AU05_r__sum_values', 'AU05_r__variance',
                    'AU05_r__standard_deviation', 'AU05_r__maximum', 'AU05_r__mean', 'AU05_r__root_mean_square', 
                    'AU06_r__sum_values', 'AU06_r__variance', 'AU06_r__standard_deviation', 'AU06_r__maximum', 
                    'AU06_r__mean', 'AU06_r__root_mean_square', 'AU07_r__sum_values', 'AU07_r__variance', 
                    'AU07_r__standard_deviation', 'AU07_r__maximum', 'AU07_r__mean', 'AU07_r__root_mean_square', 
                    'AU09_r__sum_values', 'AU09_r__variance', 'AU09_r__standard_deviation', 'AU09_r__maximum',
                    'AU09_r__mean', 'AU09_r__root_mean_square', 'AU10_r__sum_values', 'AU10_r__variance', 
                    'AU10_r__standard_deviation', 'AU10_r__maximum', 'AU10_r__mean', 'AU10_r__root_mean_square',
                    'AU12_r__sum_values', 'AU12_r__variance', 'AU12_r__standard_deviation', 'AU12_r__maximum', 
                    'AU12_r__mean', 'AU12_r__root_mean_square', 'AU14_r__sum_values', 'AU14_r__variance', 
                    'AU14_r__standard_deviation', 'AU14_r__maximum', 'AU14_r__mean', 'AU14_r__root_mean_square', 
                    'AU15_r__sum_values', 'AU15_r__variance', 'AU15_r__standard_deviation', 'AU15_r__maximum', 
                    'AU15_r__mean', 'AU15_r__root_mean_square', 'AU17_r__sum_values', 'AU17_r__variance', 
                    'AU17_r__standard_deviation', 'AU17_r__maximum', 'AU17_r__mean', 'AU17_r__root_mean_square', 
                    'AU20_r__sum_values', 'AU20_r__variance', 'AU20_r__standard_deviation', 'AU20_r__maximum', 
                    'AU20_r__mean', 'AU20_r__root_mean_square', 'AU23_r__sum_values', 'AU23_r__variance', 
                    'AU23_r__standard_deviation', 'AU23_r__maximum', 'AU23_r__mean', 'AU23_r__root_mean_square', 
                    'AU25_r__sum_values', 'AU25_r__variance', 'AU25_r__standard_deviation', 'AU25_r__maximum', 
                    'AU25_r__mean', 'AU25_r__root_mean_square', 'AU26_r__sum_values', 'AU26_r__variance', 
                    'AU26_r__standard_deviation', 'AU26_r__maximum', 'AU26_r__mean', 'AU26_r__root_mean_square', 
                    'AU45_r__sum_values', 'AU45_r__variance', 'AU45_r__standard_deviation', 'AU45_r__maximum', 
                    'AU45_r__mean', 'AU45_r__root_mean_square']

print(len(columns_ts_fresh))

102


In [14]:
columns_features =  ['VVR_1', 'VVR_2', 'AU01_r__sum_values', 'AU01_r__variance', 'AU01_r__standard_deviation', 'AU01_r__maximum', 
                    'AU01_r__mean', 'AU01_r__root_mean_square', 'AU02_r__sum_values', 'AU02_r__variance', 
                    'AU02_r__standard_deviation', 'AU02_r__maximum', 'AU02_r__mean', 'AU02_r__root_mean_square', 
                    'AU04_r__sum_values', 'AU04_r__variance', 'AU04_r__standard_deviation', 'AU04_r__maximum', 
                    'AU04_r__mean', 'AU04_r__root_mean_square', 'AU05_r__sum_values', 'AU05_r__variance',
                    'AU05_r__standard_deviation', 'AU05_r__maximum', 'AU05_r__mean', 'AU05_r__root_mean_square', 
                    'AU06_r__sum_values', 'AU06_r__variance', 'AU06_r__standard_deviation', 'AU06_r__maximum', 
                    'AU06_r__mean', 'AU06_r__root_mean_square', 'AU07_r__sum_values', 'AU07_r__variance', 
                    'AU07_r__standard_deviation', 'AU07_r__maximum', 'AU07_r__mean', 'AU07_r__root_mean_square', 
                    'AU09_r__sum_values', 'AU09_r__variance', 'AU09_r__standard_deviation', 'AU09_r__maximum',
                    'AU09_r__mean', 'AU09_r__root_mean_square', 'AU10_r__sum_values', 'AU10_r__variance', 
                    'AU10_r__standard_deviation', 'AU10_r__maximum', 'AU10_r__mean', 'AU10_r__root_mean_square',
                    'AU12_r__sum_values', 'AU12_r__variance', 'AU12_r__standard_deviation', 'AU12_r__maximum', 
                    'AU12_r__mean', 'AU12_r__root_mean_square', 'AU14_r__sum_values', 'AU14_r__variance', 
                    'AU14_r__standard_deviation', 'AU14_r__maximum', 'AU14_r__mean', 'AU14_r__root_mean_square', 
                    'AU15_r__sum_values', 'AU15_r__variance', 'AU15_r__standard_deviation', 'AU15_r__maximum', 
                    'AU15_r__mean', 'AU15_r__root_mean_square', 'AU17_r__sum_values', 'AU17_r__variance', 
                    'AU17_r__standard_deviation', 'AU17_r__maximum', 'AU17_r__mean', 'AU17_r__root_mean_square', 
                    'AU20_r__sum_values', 'AU20_r__variance', 'AU20_r__standard_deviation', 'AU20_r__maximum', 
                    'AU20_r__mean', 'AU20_r__root_mean_square', 'AU23_r__sum_values', 'AU23_r__variance', 
                    'AU23_r__standard_deviation', 'AU23_r__maximum', 'AU23_r__mean', 'AU23_r__root_mean_square', 
                    'AU25_r__sum_values', 'AU25_r__variance', 'AU25_r__standard_deviation', 'AU25_r__maximum', 
                    'AU25_r__mean', 'AU25_r__root_mean_square', 'AU26_r__sum_values', 'AU26_r__variance', 
                    'AU26_r__standard_deviation', 'AU26_r__maximum', 'AU26_r__mean', 'AU26_r__root_mean_square', 
                    'AU45_r__sum_values', 'AU45_r__variance', 'AU45_r__standard_deviation', 'AU45_r__maximum', 
                    'AU45_r__mean', 'AU45_r__root_mean_square']

print(len(columns_features))

104


# Train and test split

In total, we have 104 participants.

I started with a test size of 20%. Then there are 83 people in the train set and 21 in the test set. 
With a test size of 30#, there are 72 people in the train set and 32 in the test set. 

Naturally, we stratify on VVR_group. 

In [15]:
train, test = train_test_split(complete_dataset, test_size=0.3, random_state=123, stratify=complete_dataset['VVR_group'])

print(train.shape)
print(test.shape)

(72, 112)
(32, 112)


Unfortunately, the test set is very small with only 7 people in the high VVR condition. 

In [16]:
columns_to_drop = [ 'ID', 'Sum_12', 'Sum_4567', 'Sum_456', 'VVR_group', 'Condition', 'Date', 'Gender'] 

X_test = test.drop(columns_to_drop, axis=1)
y_test = test['VVR_group']

# Print original class distribution
print('Original dataset shape %s' % Counter(y_test))

columns_to_drop = [ 'ID', 'Sum_12', 'Sum_4567', 'Sum_456', 'VVR_group', 'Date'] 
X_test_gender = test.drop(columns_to_drop, axis=1)



Original dataset shape Counter({0: 25, 1: 7})


# X_test plus gender 

In [17]:
columns_to_drop = [ 'ID', 'Sum_12', 'Sum_4567', 'Sum_456', 'VVR_group', 'Condition', 'Date'] 

X_test_gender = test.drop(columns_to_drop, axis=1)
X_test_gender.to_csv('/Users/dionnespaltman/Desktop/V4/data/X_test_gender.csv')

# Applying SMOTE

In [18]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE

We apply SMOTE on the train data. The strategy is to make the minority as big as the majority class. 

In [19]:
columns_to_drop = [ 'ID', 'Sum_12', 'Sum_4567', 'Sum_456', 'VVR_group', 'Date'] 

X_train = train.drop(columns_to_drop, axis=1)
y_train = train['VVR_group']

# Print original class distribution
print('Original dataset shape %s' % Counter(y_train))

# Apply SMOTE to the training data with sampling strategy set to 'auto' (default)
sm = SMOTE(sampling_strategy='not majority', random_state=42, k_neighbors=5)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

# Print resampled class distribution
print('Resampled dataset shape %s' % Counter(y_train_res))

Original dataset shape Counter({0: 56, 1: 16})
Resampled dataset shape Counter({1: 56, 0: 56})


In [21]:
display(X_train_res)
X_train_res.to_csv('/Users/dionnespaltman/Desktop/V4/data/X_train_res_gender.csv')

,VVR_1,VVR_2,Condition,Gender,AU01_r__sum_values,AU01_r__variance,AU01_r__standard_deviation,AU01_r__maximum,AU01_r__mean,AU01_r__root_mean_square,...,AU26_r__standard_deviation,AU26_r__maximum,AU26_r__mean,AU26_r__root_mean_square,AU45_r__sum_values,AU45_r__variance,AU45_r__standard_deviation,AU45_r__maximum,AU45_r__mean,AU45_r__root_mean_square
0,12.000000,13.000000,2,2,-0.070220,-0.403569,-0.161650,0.428359,-0.673859,-0.387663,...,-0.537730,0.225549,-1.036133,-0.951754,-1.025247,-1.526773,-1.479742,-1.237893,-2.161830,-2.049390
1,10.000000,10.000000,3,2,2.226447,1.288304,1.124217,0.514376,0.729448,1.091318,...,0.774622,0.458331,0.368352,0.632011,1.209277,-0.225126,-0.001447,0.619766,-0.202961,-0.127515
2,8.000000,8.000000,3,1,-0.328541,-0.286339,-0.049149,0.514376,-0.191466,-0.152619,...,0.539267,0.408449,0.316323,0.450683,-0.834081,-0.909004,-0.642000,0.297037,-1.125698,-0.981679
3,10.000000,8.000000,1,2,-0.179816,1.024130,0.956659,0.514376,0.292720,0.824388,...,0.891064,0.458331,0.480862,0.766552,-0.208294,0.810246,0.749643,0.257680,0.422548,0.743912
4,8.000000,8.000000,1,2,-0.687261,-0.791730,-0.584920,0.514376,-0.763173,-0.767495,...,-0.558621,0.366881,-0.858611,-0.877156,-0.419249,-0.490562,-0.230352,-0.340549,-0.573360,-0.444615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,15.208788,9.791212,1,1,-0.487217,-0.225515,0.005625,0.181101,-0.528458,-0.206212,...,-0.226106,0.412286,-0.469718,-0.456650,-0.012510,0.388434,0.334707,0.252315,0.284033,0.322442
108,16.925449,10.850899,1,1,-0.629858,-0.610840,-0.380021,0.392113,-0.678192,-0.572561,...,-0.168894,0.458331,-0.072297,-0.205383,-0.447768,-0.289515,-0.066093,0.619766,-0.590136,-0.295949
109,10.986887,11.973774,2,2,1.755451,0.208069,0.373092,0.514376,0.195169,0.312097,...,0.398382,0.458331,0.303690,0.353698,3.025754,1.353332,1.082597,0.619766,1.220385,1.279031
110,15.861224,10.544490,2,2,-0.583734,-0.648807,-0.418202,0.436879,-0.696419,-0.609791,...,0.646099,0.458331,0.929210,0.852713,-0.537762,-0.516713,-0.266038,0.470967,-0.855955,-0.556206


# Adding class weights

I will add class weights to my models, because of my inbalanced data set. 
https://medium.com/@ravi.abhinav4/improving-class-imbalance-with-class-weights-in-machine-learning-af072fdd4aa4 

Results: 
{0: 0.6428571428571429, 1: 2.25}

In [25]:
import numpy as np

def calculate_class_weights(y):
    unique_classes, class_counts = np.unique(y, return_counts=True)
    total_samples = len(y)
    class_weights = {}

    for class_label, class_count in zip(unique_classes, class_counts):
        class_weight = total_samples / (2.0 * class_count)
        class_weights[class_label] = class_weight

    return class_weights

# Assuming 'y' contains the class labels (0s and 1s) for the binary classification problem
class_weights = calculate_class_weights(y_train)
print("Class weights:", class_weights)


Class weights: {0: 0.6428571428571429, 1: 2.25}


# Save X_train_res, y_train_res, X_test, y_test

In [26]:
print(y_test)

64     0
39     1
47     0
90     1
94     0
84     0
85     0
44     0
43     0
62     1
58     0
76     0
41     0
9      0
103    1
50     0
38     0
56     0
57     0
20     1
67     0
51     1
48     0
74     1
82     0
3      0
33     0
30     0
99     0
21     0
70     0
60     0
Name: VVR_group, dtype: int64


In [15]:
X_train_res.to_csv('/Users/dionnespaltman/Desktop/V4/data/X_train_res.csv')
y_train_res.to_csv('/Users/dionnespaltman/Desktop/V4/data/y_train_res.csv')
X_test.to_csv('/Users/dionnespaltman/Desktop/V4/data/X_test.csv')
y_test.to_csv('/Users/dionnespaltman/Desktop/V4/data/y_test.csv')

# X_train_12 and X_test_12 

In this set we want to drop all the action units. 

In [27]:
columns = columns_ts_fresh

print(X_train.shape)
print(X_test.shape)

X_train_12 = X_train.drop(columns=columns)
X_test_12 = X_test.drop(columns=columns)

print(X_train_12.shape)
print(X_test_12.shape)

(72, 104)
(32, 104)
(72, 2)
(32, 2)


In [30]:
X_train_12.to_csv('/Users/dionnespaltman/Desktop/V4/data/X_train_12.csv')
X_test_12.to_csv('/Users/dionnespaltman/Desktop/V4/data/X_test_12.csv')
y_train.to_csv('/Users/dionnespaltman/Desktop/V4/data/y_train_12.csv')